#### import

In [4]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.select import Select
import time

import webdriver_manager
from webdriver_manager.chrome import ChromeDriverManager


import warnings
warnings.filterwarnings('ignore')


%matplotlib inline

### API 청약정보 수집

- 기간: 2020-01-01 ~ 2020-09-30
- 지역: 서울, 경기, 인천
- 수집: 아파트명, 지역, 도로명주소, 세대수, 시행사, 시공사, 세부정보(URL)

In [5]:
key = ''
시작일 = '2020-01-01'
마지막일 = '2023-09-30'
지역들 = ['서울','경기','인천']

In [6]:
data_dict = {
    '타입': [],
    '아파트명': [],
    '지역': [],
    '도로명주소': [],
    '세대수': [],
    '시행사': [],
    '시공사': [],
    '세부정보': []
}


for 지역 in 지역들:
    url = f'https://api.odcloud.kr/api/ApplyhomeInfoDetailSvc/v1/getAPTLttotPblancDetail?page=1&perPage=200&cond[SUBSCRPT_AREA_CODE_NM::EQ]={지역}&cond[RCRIT_PBLANC_DE::LTE]={마지막일}&cond[RCRIT_PBLANC_DE::GTE]={시작일}&serviceKey={key}'

    res = requests.get(url)
    data = res.json()
    
    # json에서 원하는 데이터 저장
    for i in range(data['currentCount']):
        data_dict['타입'].append(data['data'][i]['HOUSE_SECD_NM'])
        data_dict['아파트명'].append(data['data'][i]['HOUSE_NM'])
        data_dict['지역'].append(data['data'][i]['SUBSCRPT_AREA_CODE_NM'])
        data_dict['도로명주소'].append(data['data'][i]['HSSPLY_ADRES'])
        data_dict['세대수'].append(data['data'][i]['TOT_SUPLY_HSHLDCO'])
        data_dict['시행사'].append(data['data'][i]['BSNS_MBY_NM'])
        data_dict['시공사'].append(data['data'][i]['CNSTRCT_ENTRPS_NM'])
        data_dict['세부정보'].append(data['data'][i]['PBLANC_URL'])

In [7]:
df = pd.DataFrame(data_dict)

### selenium으로 세부정보 수집

- 수집: 주택형, 주택공급면적, 공급세대수, 공급금액(최고가 기준), 입주예정월

In [61]:
data_dict2 = {
    '타입': [],
    '아파트명': [],
    '지역': [],
    '도로명주소': [],
    '세대수': [],
    '시행사': [],
    '시공사': [],
    '주택형': [],
    '주택공급면적': [],
    '공급세대수': [],
    '공급금액(최고가 기준)': [],
    '입주예정월': []
}

# service = ChromeService(executable_path=ChromeDriverManager().install())
options = Options()
# 크롬을 숨기기
options.add_argument("--headless")
options.add_argument('--blink-settings=imagesEnabled=false')
# 데이터를 많이 요청할경우 agent라는걸 알려주는 메시지를 보냄
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.199 Safari/537.36")
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 10)


for num, url in enumerate(df['세부정보']):
    driver.get(url)
    table = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.tbl_scroll')))

    # 테이블에 데이터 크롤링
    tbody = table.find_element(By.TAG_NAME, 'tbody') 
    rows = tbody.find_elements(By.TAG_NAME, 'tr') 


    table_xpath = "//*[@id='printArea']/table[2]"
    table2 = driver.find_element(By.XPATH, table_xpath)
    tbody2 = table2.find_element(By.TAG_NAME, 'tbody') 
    rows2 = tbody2.find_elements(By.TAG_NAME, 'tr') 


    li_xpath = "//*[@id='printArea']/ul[3]/li[1]"
    li_element = driver.find_element(By.XPATH, li_xpath)
    li_text = li_element.text[-7:]
    li_text = f'{li_text[:4]}-{li_text[5:]}-01'

    result_list = []
    result_list2 = []
    
    # 테이블 데이터를 리스트로 저장
    for row in rows[:-1]:
        cells = row.find_elements(By.TAG_NAME, 'td')
        result_list.append([cell.text for cell in cells])
        
    for row2 in rows2:
        cells = row2.find_elements(By.TAG_NAME, 'td')
        result_list2.append([cell.text for cell in cells])    
        
    # 기존 데이터와 리스트의 데이터 추가
    for idx, val in enumerate(result_list):
        if idx == 0:
            val.pop(0)
        data_dict2['타입'].append(df['타입'][num])
        data_dict2['아파트명'].append(df['아파트명'][num])
        data_dict2['지역'].append(df['지역'][num])
        data_dict2['도로명주소'].append(df['도로명주소'][num])
        data_dict2['세대수'].append(df['세대수'][num])
        data_dict2['시행사'].append(df['시행사'][num])
        data_dict2['시공사'].append(df['시공사'][num])
        
        # 데이터 추가
        data_dict2['주택형'].append(val[0])
        data_dict2['주택공급면적'].append(val[1])
        data_dict2['공급세대수'].append(val[4])

    for val in result_list2:
        data_dict2['공급금액(최고가 기준)'].append(val[1])

    for i in range(idx+1):
        data_dict2['입주예정월'].append(li_text)
        
    
    time.sleep(3)

In [63]:
df2 = pd.DataFrame(data_dict2)

In [68]:
df2.to_csv('2020년9월.csv', index=False, encoding='cp949')

이름이 달라(띄어쓰기, 완공후 아파트명 변경, 주소와 겹쳐있는 등) 건물 정보(건폐율, 전용율, 주차대수 등)는 네이버 부동산 크롤링 데이터 또는 호갱노노에서 검색을 통해 채워 넣음